<a href="https://colab.research.google.com/github/russmendez/DS_notebooks/blob/master/Excel_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Drive and Install Libraries

In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandasql
!pip install openpyxl

  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26784 sha256=b66021fc4c08a133c48a245428eccd336104ad8957f07cd7dcb1b1aa429788b0
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


# Generate a template 

In [1]:
#-------------------------------------------------------------------------------
# Load libraries and hardcode file path, create new Excel sheet

import openpyxl
from openpyxl import Workbook
from openpyxl.utils import FORMULAE
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Color, Alignment, Border, Side, colors, PatternFill
from openpyxl.formatting.rule import FormulaRule

workbook = Workbook()
sheet = workbook.active

filename = 'xl_file.xlsx' # change to desired file name

workbook.save(filename=filename)
!cp xl_file.xlsx 'drive/My Drive/python_for_data_science/'

#-------------------------------------------------------------------------------
# Date input

date_input = {
              'Month' : None,
              'Day': None
              }

if date_input['Month'] is None:
  date_input['Month'] = input('Enter month as 1-12: ')
  
if date_input['Day'] is None:
  date_input['Day'] = input('Enter day as 1-31: ')

Enter month as 1-12: 5
Enter day as 1-31: 15


###template generation

The requirements for the template are as follows:

1. There will be 15 categorical columns before the numeric columns. The first column with be the index from the original df and will not need a filter but the rest will. 

2. There will need to be 6 numeric columns for each month. The number of columns should depend on when the template is generated, going forward 12 months to the end of 2021.

3. Depending on if its before or after the 15th of the month that month will be considered either historic or forcasting. Forcasting months will need one of two types of SUBTOTAL functions applied, explained in the code below. Historic months will not need any SUBTOTAL functions.

4. Shade every other month and highlight the override column. 



In [2]:
#-------------------------------------------------------------------------------
# Formatting. openpyxl requires the style argument to be stored as an array. Cells cannot be referenced directly for styling

from openpyxl.styles import Font, Color, Alignment, Border, Side, colors, PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import FormulaRule



purple_text = Font(color='00FF00FF', bold=True)

bold_font = Font(bold=True)
center_aligned_text = Alignment(horizontal="center", shrinkToFit=True)
single_border_side = Side(border_style='thin')
single_square_border = Border(top = single_border_side,
                              right = single_border_side,
                              bottom = single_border_side,
                              left = single_border_side)
double_border_side = Side(border_style="double")
double_square_border = Border(top=double_border_side,
                      right=double_border_side,
                      bottom=double_border_side,
                      left=double_border_side)

greyFill = PatternFill(start_color='00C0C0C0', fill_type='solid')

fillBlue = PatternFill(start_color='0099CCFF', fill_type='solid')

fillGreen = PatternFill(start_color='0000FF00', fill_type='solid')

#-------------------------------------------------------------------------------
# Subcolumns header creation. These titles must be hard coded. This will allow for the creation of any number of subcolumns. They just need to be added to the lists. 

months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
# This is a template for the five coumns needed for each month
sub_columns_21 = ["XXX_'20", "F_XXX_'21", "ABC_XXX_'21", "O_XXX_'21", "XXX_'21_%CHG", "XXX_'21_%DELTA"]
sub_columns_22 = ["XXX_'21", "F_XXX_'22", "ABC_XXX_'21", "O_XXX_'22", "XXX_'22_%CHG", "XXX_'22_%DELTA"]

sub_columns_titles = ['PRIOR', 'FORECAST', 'ABC', 'OVERIDE', '% CHG', '% DELTA']


# If the input date is >= 15 than the current month is considered 'historical.' Because the index is set back 1, no change will 
# need to be made as the right hand of a range is inclusive. i.e. if its 4/25 then April will be 'historical' and 
# so 4 will be the righ hand bound on the range.

if int(date_input['Day']) >= 15:
  col_list = []
  title_col_list = []
  x = int(date_input['Month'])

  for i in range(0,x):
    new_test = [s.replace('XXX', f'{months[i]}') for s in sub_columns_21]
    col_list.append(new_test)
    title_col_list.append(sub_columns_titles)

  for i in range(x,12):
    new_test = [s.replace('XXX', f'{months[i]}') for s in sub_columns_21]
    col_list.append(new_test)
    title_col_list.append(sub_columns_titles)

  for i in range(0,6):
      new_test = [s.replace('XXX', f'{months[i]}') for s in sub_columns_22]
      col_list.append(new_test)
      title_col_list.append(sub_columns_titles)
  

# If the input date is before the 15th then the month is considered 'forcasting' and therefore is not included into the 'historical month.
# This is managed by removing 1 from the date thereby making the previous month the inclusive right bound on the range.

if int(date_input['Day']) < 15:
  col_list = []
  title_col_list = []
  x = int(date_input['Month']) - 1

  for i in range(0,x):
    new_test = [s.replace('XXX', f'{months[i]}') for s in sub_columns_21]
    col_list.append(new_test)
    title_col_list.append(sub_columns_titles)

  for i in range(x,12):
    new_test = [s.replace('XXX', f'{months[i]}') for s in sub_columns_21]
    col_list.append(new_test)
    title_col_list.append(sub_columns_titles)

  for i in range(0,6):
      new_test = [s.replace('XXX', f'{months[i]}') for s in sub_columns_22]
      col_list.append(new_test)
      title_col_list.append(sub_columns_titles)


#-------------------------------------------------------------------------------
# Adding the subcolumns headers to sheet/ formatting rows 2 and 6.

new_list = [item for sublist in col_list for item in sublist] # makes list of list into a single list for easier coding
new_title_list = [item for sublist in title_col_list for item in sublist]

# This code creates a row and assumes there will be a header above. So min_row=1 will place the row at row 2 on the excell sheet.
# Functionally, this means that min_row can be used to indicate how many rows need to be above the input line.

for row in sheet.iter_rows(min_row = 1, max_row = 1): # Top row
  sheet.append(new_list)

for row in sheet.iter_rows(min_row = 5, max_row = 5): # header for bottom row
  sheet.append(new_title_list)

for row in sheet.iter_rows(min_row = 6, max_row = 6): # bottom row
  sheet.append(new_list)

for cell in sheet['2:2']: # This is done here before the categorical columns are added to prevent the whole upper section from having borders.
  cell.font = bold_font
  cell.border = single_square_border
  cell.alignment = center_aligned_text

for cell in sheet['6:6']:
  cell.font = bold_font
  cell.border = single_square_border
  cell.alignment = center_aligned_text

for cell in sheet['1:1']:
  cell.font = bold_font
  cell.border = single_square_border
  cell.alignment = center_aligned_text
#-------------------------------------------------------------------------------
# Creating the categorical columns and formatting them. 

# IMPORTANT NOTE: These cells needed to be hard coded individually. Because there is no list of categorical columns,
# it was not possible to automated some of the following sections. Instead I refer to the number of columns so if the number of columns change
# a careful revision of the code will be needed to ensure it functions correctly.

#ST	H1	H2	H5	H4	H5	H6	H7	H8	H9	DESC	CGY1	CGY2	CGY3
sheet.insert_cols(idx=1, amount=14) # category columns
sheet['A7'] = 'ST'
sheet['B7'] = 'H1'
sheet['C7'] = 'H2'
sheet['D7'] = 'H5'
sheet['E7'] = 'H4'
sheet['F7'] = 'H5'
sheet['G7'] = 'H6'
sheet['H7'] = 'H7'
sheet['I7'] = 'H8'
sheet['J7'] = 'H9'
sheet['K7'] = 'DESC'
sheet['L7'] = 'CGY1'
sheet['M7'] = 'CGY2'
sheet['N7'] = 'CGY3'

sheet.insert_cols(idx=1, amount=1) # empty column for original index

for i in range(0,14): # grey formatting
  letter = get_column_letter(i+2)
  sheet[f'{letter}7'].fill = greyFill

#-------------------------------------------------------------------------------
# Adding the SUBTOTAL function to the proper month, adding filter to categorical columns, and formatting row 7.

# x is the first forecast month. (x*(number of rows generated per month) +1) will start the SUBTOTAL in the correct position. Plus 1 because of the difference in indexing
# Using sheet.max_column makes the code end 1 short because the right hand of range() is non-inclusive.
# In addition, because we add 15 columns to the left later we also need to account for that when determining the range otherwise the SUBTOTAL will count the wrong columns.

for i in range((((x-1)*6) +16), (sheet.max_column +1)): #  SUBTOTAL row
  letter = get_column_letter(i)
  n = f'=SUBTOTAL(9, {letter}8:{letter}50000)'
  sheet[f"{letter}3"] = n
  sheet[f"{letter}3"].font = purple_text

for i in range((((x-1)*6) +20), (sheet.max_column +1),6): #  SUBTOTAL row
  letter = get_column_letter(i)
  n = f'=SUBTOTAL(1, {letter}8:{letter}50000)'
  sheet[f"{letter}3"] = n
  sheet[f"{letter}3"].font = purple_text

for i in range((((x-1)*6) +21), (sheet.max_column +1),6): #  SUBTOTAL row
  letter = get_column_letter(i)
  n = f'=SUBTOTAL(1, {letter}8:{letter}50000)'
  sheet[f"{letter}3"] = n
  sheet[f"{letter}3"].font = purple_text

sheet.auto_filter.ref = "B7:O50000" #add filter

for cell in sheet['7:7']: # This is done after the categorical columns are added so the entire row will have borders.
  cell.font = bold_font
  cell.border = single_square_border
  cell.alignment = center_aligned_text

#-------------------------------------------------------------------------------
# Formatting large merged cell - no longer required. Keep for reference

#sheet.merge_cells(start_row=5, start_column=2, end_row=6, end_column=15) # Merge the block above the category columns.
#sheet['B5'].border = single_square_border # format border

#-------------------------------------------------------------------------------
# Adding/formatting month headers. The numbers used are based on the categorical columns being created beforehand. 

month_titles = ['JANUARY', 'FEBURARY', 'MARCH',
                'APRIL', 'MAY', 'JUNE', 
                'JULY', 'AUGUST', 'SEPTEMBER',
                'OCTOBER', 'NOVEMBER', 'DECEMBER']

month_header_count = 0

for i in range(16, (sheet.max_column + 1), 6):
  start_letter = get_column_letter(i)
  end_letter = get_column_letter(i+5)
  
  
  if month_header_count < 12:
      month_header = month_titles[month_header_count]
      #print(f'{start_letter}, {end_letter}, {month_header}')
      sheet.merge_cells(f'{start_letter}1:{end_letter}1')
      sheet[f'{start_letter}1'] = month_header #to fill a merged cell, refer to the uper left most cell in the block
      sheet[f'{start_letter}1'].font = bold_font
      #sheet[f'{start_letter}1'].border = double_square_border
      sheet[f'{start_letter}1'].alignment = center_aligned_text

      sheet.merge_cells(f'{start_letter}5:{end_letter}5')
      sheet[f'{start_letter}5'] = month_header
      sheet[f'{start_letter}5'].font = bold_font
      #sheet[f'{start_letter}5'].border = double_square_border
      sheet[f'{start_letter}5'].alignment = center_aligned_text 

  if month_header_count > 11:
      month_header_count = month_header_count - 12
      month_header = month_titles[month_header_count]
      #print(f'{start_letter}, {end_letter}, {month_header}')
      sheet.merge_cells(f'{start_letter}1:{end_letter}1')
      sheet[f'{start_letter}1'] = month_header
      sheet[f'{start_letter}1'].font = bold_font
      #sheet[f'{start_letter}1'].border = double_square_border
      sheet[f'{start_letter}1'].alignment = center_aligned_text

      sheet.merge_cells(f'{start_letter}5:{end_letter}5')
      sheet[f'{start_letter}5'] = month_header
      sheet[f'{start_letter}5'].font = bold_font
      #sheet[f'{start_letter}5'].border = double_square_border
      sheet[f'{start_letter}5'].alignment = center_aligned_text

  month_header_count = month_header_count + 1

for i in range(16, (sheet.max_column + 1), 12): # coloring every other cell.
  start_letter = get_column_letter(i)
  sheet[f'{start_letter}1'].fill = fillBlue
  sheet[f'{start_letter}5'].fill = fillBlue

#-------------------------------------------------------------------------------
# Adding/formatting month headers

for i in range(19, (sheet.max_column + 1), 6): 
  for row in sheet.iter_cols(min_col=i, max_col=i, min_row=1, max_row=5000): # If the code needs to run faster, limit the max_row 
    for cell in row:
      cell.fill = fillGreen

#-------------------------------------------------------------------------------
# Save template. Hardcode file path.
workbook.save(filename=filename)
!cp xl_file.xlsx 'drive/My Drive/python_for_data_science/'

# Filling template

In [4]:
#-------------------------------------------------------------------------------
# Load the data into the template.

import pandas as pd

input = 'drive/My Drive/python_for_data_science/Copy_input_from_python_process.xlsx'
df = pd.read_excel(input, header=None) # Make sure there will be no formulas in the input data.
df = df.drop(axis=0, index=0) # drop the header to properly fill the cells.
df.iloc[:,0] = df.iloc[:,0].astype(int)
#df
for i in range(0, len(df.columns)):
  row_counter = 8
  try:
    values = list(df.iloc[:,i])
    letter = get_column_letter(i+1)
    for ii in values:

      sheet[f'{letter}{row_counter}'] = ii
      row_counter = row_counter + 1
  except Exception:
    pass

#-------------------------------------------------------------------------------
# Save template. Hardcode file path.
workbook.save(filename=filename)
!cp xl_file.xlsx 'drive/My Drive/python_for_data_science'

#[Final Product](https://docs.google.com/spreadsheets/d/1-JOvE0-lC53LzzKHeP2Ol1wukiWFX1Rx/edit#gid=155305879) 